# Webscraping from cek-ongkir.com

In [1]:
# LIBRARIES

from selenium.webdriver import Chrome
import pandas as pd
import gspread

In [2]:
# GSHEET PYTHON INITIALIZATION

srv_acc = gspread.service_account(filename='scraping-cekongkir-7a1ffd441444.json')
sheet = srv_acc.open("Hari-Budiarto_Python")
wrksh_read = sheet.worksheet('City_list')
wrksh_write = sheet.worksheet('Question2')

In [7]:
# GET OD LIST FROM GSHEET

od_list = wrksh_read.get_all_values()

print('PREVIEW ORIGIN DESTINATION LIST FROM GSHEET')
print(od_list[0])

PREVIEW ORIGIN DESTINATION LIST FROM GSHEET
['Yogyakarta', 'Kota Yogyakarta', 'Pagar Alam', 'Kota Pagar Alam']


In [8]:
# SCRAPING AND PUT RESULT INTO LIST

data = []
driver = Chrome()

for row in od_list:
    
    # Unpack od list
    o, o_complete, d, d_complete = row

    
    # Open the website
    driver.get('https://cek-ongkir.com')
    driver.maximize_window()
    driver.implicitly_wait(10)
    
    
    # Fill origin form
    orig = driver.find_element_by_xpath('//input[@name="origin"]')
    orig.clear()
    orig.send_keys(o)

    for orig_complete in driver.find_elements_by_xpath('//ul[@id="ui-id-1"]//div'):
        if orig_complete.text == o_complete:
            driver.execute_script("arguments[0].click();", orig_complete)
            break


    # Fill dest form        
    dest = driver.find_element_by_xpath('//input[@name="destination"]')
    dest.clear()
    dest.send_keys(d)
    
    for dest_complete in driver.find_elements_by_xpath('//ul[@id="ui-id-2"]//div'):
        if dest_complete.text == d_complete:
            driver.execute_script("arguments[0].click();", dest_complete)
            break

            
    # Find result        
    cek_ongkir = driver.find_element_by_xpath('//button[@id="send"]')
    driver.execute_script("arguments[0].click();", cek_ongkir)

    
    # Scrape the result and put in list
    results = driver.find_elements_by_xpath('//table[@id="result-ongkir"]')

    for r in results:
        courier = r.find_element_by_xpath('.//td[1]').text
        price = r.find_element_by_xpath('.//td[2]').text
        duration = r.find_element_by_xpath('.//p').text
        new_row = {'courier': courier, 'price': price,'duration': duration,
                   'origin': o_complete,'destination': d_complete}
        data.append(new_row)



In [15]:
# PREPARE DATA FOR EXPORT

df = pd.DataFrame.from_records(data)
# df.rename(columns = {'courier':'3PL','price':'Ongkir','duration':'ETA',
#                     'origin':'From','destination':'To'}, inplace = True)

df = df.loc[df['courier'] == 'Jalur Nugraha Ekakurir (JNE)\n[REG ] Layanan Reguler'] 

print('PREVIEW SCRAPING RESULT')
df.head()

PREVIEW SCRAPING RESULT


,courier,price,duration,origin,destination
1,Jalur Nugraha Ekakurir (JNE)\n[REG ] Layanan R...,Rp.53000,4-5 hari,Kota Yogyakarta,Kota Pagar Alam
3,Jalur Nugraha Ekakurir (JNE)\n[REG ] Layanan R...,Rp.124000,3-4 hari,Kota Tual,Kota Padang Sidempuan
5,Jalur Nugraha Ekakurir (JNE)\n[REG ] Layanan R...,Rp.92000,3-4 hari,Kota Tomohon,Kota Padang Panjang
7,Jalur Nugraha Ekakurir (JNE)\n[REG ] Layanan R...,Rp.117000,2-3 hari,Kota Tidore Kepulauan,Kota Padang
9,Jalur Nugraha Ekakurir (JNE)\n[REG ] Layanan R...,Rp.78000,2-3 hari,Kota Ternate,Kota Mojokerto


In [17]:
# WRITE DATA TO GSHEET

wrksh_write.update('A1',[['3PL','Ongkir','ETA','From','To']])
wrksh_write.update('A2',df.values.tolist())

/Users/haribudiarto/anaconda3/lib/python3.10/site-packages/gspread/worksheet.py:1046: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  warnings.warn(


{'spreadsheetId': '1MnRgKztsYs-JkUm-EelPwEDwQllJnGg6HjyIJY9E7_c',
 'updatedRange': 'Question2!A2:E48',
 'updatedRows': 47,
 'updatedColumns': 5,
 'updatedCells': 235}